# Building a house (Harvard Business Review, 1963)

Several tasks must be completed in order to build our house. Each task has a duration, tasks may be worked on simultaneously, but there is also a precedence relation. Some tasks can only be started once other tasks have been completed. The following table shows each task, it's duration, and the tasks that must be completed before it starts. How fast can the house be built?
![alt text](https://hbr.org/resources/images/article_assets/hbr/6309/63508_B.gif)

Let's model this as the longest path through the graph.

In [2]:
#Reuse the same data as last time

#This array stores the task names (:a, :b, ..., :x)
tasks = []
for i = 'a':'x'
    #Can convert back to string using string(sym), i.e. string(:hello) returns "hello"
    push!(tasks, Symbol(i))
end

#Dictionary to store the project durations
dur = [0, 4, 2, 4, 6, 1, 2, 3, 2, 4, 10, 3, 1, 2, 3, 2, 1, 1, 2, 3, 1, 2, 5, 0]
duration = Dict(zip(tasks,dur))

#Dictionary to store the projects that a given project depends on (ancestors)
pre = ( [], [:a], [:b], [:c], [:d], [:c], [:f], [:f], [:d], [:d,:g], [:i,:j,:h], [:k],
    [:l], [:l], [:l], [:e], [:p], [:c], [:o,:t], [:m,:n], [:t], [:q,:r], [:v], [:s,:u,:w])
pred = Dict(zip(tasks,pre));

numedges = 0
for i in tasks
    for j in pred[i]
        numedges = numedges + 1
    end
end

#Dictionaries that map from tasks to numbers and vice versa
taskstonum = Dict(zip(tasks,1:size(tasks,1)))
numtotasks = Dict(zip(1:size(tasks,1),tasks))

#Dictionaries to remember where an edge started and an edge ended, respectively
edgestart = Dict()
edgeend = Dict()

#Initialize the incidence matrix as zeros, with number of rows = number of tasks
#and number of columns = number of edges
A = zeros(size(tasks,1),numedges)
edgenum = 1
#Loop over all combinations of tasks and predecessors
for i in tasks
    for j in pred[i]
        #Convert the symbols for tasks and predecessors to numbers
        ni = taskstonum[i]
        nj = taskstonum[j]
        #Update this edges column with an edge from j to i
        A[nj,edgenum] = 1
        A[ni,edgenum] = -1
        #Record what is the start and end of this edge in the dictionaries
        edgestart[edgenum] = nj
        edgeend[edgenum] = ni
        #Counter to keep place for number of edges
        edgenum = edgenum+1
    end
end

#1 flow at top of graph (source), -1 flow at bottom (sink)
b = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1];
A

24×31 Matrix{Float64}:
  1.0   0.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
 -1.0   1.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0  -1.0   1.0   0.0   1.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0  -1.0   1.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0  -1.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0  -1.0   1.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0  -1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  ⋮                             ⋮    ⋱   ⋮                             ⋮
  0.0   0.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      1.0   0.0   0.0   0.0   0.0   0.0
  0.

In [3]:
using JuMP
using HiGHS

m = Model(HiGHS.Optimizer)

#Edges from node j to node i
@variable(m, 1 >= edge[i in 1:numedges] >= 0)

@constraint(m, flow, A*edge .== b)

@objective(m, Max, sum(dur[edgestart[i]]*edge[i] for i in 1:numedges))

print(m)

Max 4 edge[2] + 2 edge[3] + 4 edge[4] + 2 edge[5] + edge[6] + edge[7] + 4 edge[8] + 4 edge[9] + 2 edge[10] + 2 edge[11] + 4 edge[12] + 3 edge[13] + 10 edge[14] + 3 edge[15] + 3 edge[16] + 3 edge[17] + 6 edge[18] + 2 edge[19] + 2 edge[20] + 3 edge[21] + 3 edge[22] + edge[23] + 2 edge[24] + 3 edge[25] + edge[26] + edge[27] + 2 edge[28] + 2 edge[29] + edge[30] + 5 edge[31]
Subject to
 flow : edge[1] = 1
 flow : -edge[1] + edge[2] = 0
 flow : -edge[2] + edge[3] + edge[5] + edge[20] = 0
 flow : -edge[3] + edge[4] + edge[8] + edge[9] = 0
 flow : -edge[4] + edge[18] = 0
 flow : -edge[5] + edge[6] + edge[7] = 0
 flow : -edge[6] + edge[10] = 0
 flow : -edge[7] + edge[13] = 0
 flow : -edge[8] + edge[11] = 0
 flow : -edge[9] - edge[10] + edge[12] = 0
 flow : -edge[11] - edge[12] - edge[13] + edge[14] = 0
 flow : -edge[14] + edge[15] + edge[16] + edge[17] = 0
 flow : -edge[15] + edge[23] = 0
 flow : -edge[16] + edge[24] = 0
 flow : -edge[17] + edge[21] = 0
 flow : -edge[18] + edge[19] = 0
 flow : 

In [4]:
optimize!(m)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
10 rows, 17 cols, 34 nonzeros
7 rows, 14 cols, 28 nonzeros
5 rows, 12 cols, 24 nonzeros
3 rows, 7 cols, 12 nonzeros
Presolve : Reductions: rows 3(-21); columns 7(-24); elements 12(-50)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          3    -3.4000000000e+01 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 3
Objective value     :  3.4000000000e+01
HiGHS run time      :          0.00


In [5]:
@show objective_value(m)

for i in 1:numedges
    if(value(edge[i]) > 10^-9)
        println("The edge from ", numtotasks[edgestart[i]], " to ", numtotasks[edgeend[i]], " is part of the longest path.")
    end
end

objective_value(m) = 34.0
The edge from a to b is part of the longest path.
The edge from b to c is part of the longest path.
The edge from c to d is part of the longest path.
The edge from d to j is part of the longest path.
The edge from j to k is part of the longest path.
The edge from k to l is part of the longest path.
The edge from l to n is part of the longest path.
The edge from t to s is part of the longest path.
The edge from n to t is part of the longest path.
The edge from s to x is part of the longest path.
